In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy')

import gqcpy

# Import numpy
import numpy as np

# ONV path manipulation

An ONV, or occupation number vector, is a crucial element in Full CI calculations. In this example notebook we will show you how you can use gqcpy to create and manipulate ONV paths. 

## The ONV basis

In this example we will work with a spin unresolved ONV basis, meaning that there is no distinction between alpha and beta electrons.

In [2]:
# Create an ONV basis of 3 electrons and 5 possible orbitals
onv_basis = gqcpy.SpinUnresolvedONVBasis(5, 3)

The ONV Basis itself has some useful functions we can call.

In [3]:
# Get the weight of the arc starting at coordinate (p, n)
onv_basis.arcWeight(0,0)

0

In [4]:
# Get the weight of the vertex at coordinate (p, n)
onv_basis.vertexWeight(3, 2)

3

In [5]:
# Get the dimension of the ONV basis
onv_basis.calculateDimension(5, 3)

10

## A specific ONV and its path

We can create an ONV and it's path in two differenbt ways.

In [6]:
# Create an ONV
# The indices determine where the occupied orbitals should be: "1" in the bitstring
onv = gqcpy.SpinUnresolvedONV.FromOccupiedIndices([0, 2, 3], 5)

# from string is read right to left
onv_2 = gqcpy.SpinUnresolvedONV.FromString("01101")

In [7]:
# Create the ONV path
ONV_path = gqcpy.ONVPath(onv_basis, onv)
ONV_path_2 = gqcpy.ONVPath(onv_basis, onv_2)

Both of these commands create the exact same ONV and path. From here on out, we will continue with the one named `onv`. The ONV created looks as follows:

In [8]:
ONV_path.address()

2

In [9]:
ONV_path_2.address()

2

In [10]:
ONV_path.sign()

1

In [11]:
ONV_path.annihilate(0,0)

In [12]:
ONV_path.address()

2

In [13]:
ONV_path.nextCreationIndex()

1

In [15]:
ONV_path.create(1,0)

In [16]:
ONV_path.address()

3